In [53]:
import pandas as pd
import time
import numpy as np
from tqdm import tqdm_notebook as tqdm
from numpy import dot, array
import scipy
from scipy import sparse
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.datasets import make_classification
from sklearn.datasets import make_blobs
from sklearn.datasets import make_gaussian_quantiles
import cvxopt
import math

In [8]:
def dimensionality_reduction(X_data,Y_label,save=False):
    meanVector = np.mean(X_data,axis = 0)
    centeredX_data = X_data - meanVector
    sdVector = np.std(centeredX_data.astype(float),axis = 0)
    for index,value in enumerate(sdVector):
      if value <= 1e-5:
          sdVector[index] = 1.0  
        
    centeredX_data /= sdVector
    
    covX_data = np.array([centeredX_data[0,:]]).T@ np.array([centeredX_data[0,:]])
    for i in range(1,len(centeredX_data)):
        covX_data += np.array([centeredX_data[i,:]]).T@ np.array([centeredX_data[i,:]])
    covX_data /= len(covX_data)

    eigenValues, eigenVectors = np.linalg.eig(covX_data)
    idx = eigenValues.argsort()[::-1] 
    eigenVectors = eigenVectors[:,idx]

    neededEigenVectors = []
    for idx, val in enumerate(eigenValues):
        if (sum(eigenValues[0:idx])/sum(eigenValues))>0.7:
            neededEigenVectors = eigenVectors[:,0:idx]
            break

    reconstructX_data = centeredX_data@neededEigenVectors
    print('reduced dimensions are ')
    print(reconstructX_data.shape)
    return reconstructX_data

In [99]:
def normsquare(a,b):    
    a_min_b = a - b
    return sum((a - b)**2)
    #return (np.sum(np.square(a_min_b)))
    #return np.einsum('ij,ij->i', a_min_b, a_min_b)

In [104]:
class SMCSVM:
    def __init__(self):
        self.kernel = 'rbf'
        self.C=10
        self.degree = 1
        self.sigma=1.0
        self.alphas2=None
        self.fullXmatrix=None
        self.IntermXmatrix=None
        self.train_X_data = None
        self.num_classes = None
    
    def fit(self, X_data, Y_label,C=10,kernel='rbf',degree=1,sigma=1.0):
        self.kernel = kernel
        self.C= C
        self.degree = degree
        self.sigma= sigma 
        self.train_X_data = X_data
        X = X_data
        trainLabelVector = Y_label
        #fullXmatrix =[]
        num_classes = len(np.unique(Y_label))
        self.num_classes = num_classes
        #print(trainLabelVector.shape)

        fullXmatrix = np.zeros(((X_data.shape[0]*(num_classes-1)),num_classes*((X_data.shape[1])+1)))

        #for i in tqdm(range(0,(X_data.shape[0]*(num_class-1)))):

        count5 = 0
        print('\n Calculating fullXmatrix')
        for i in tqdm(range(0,X.shape[0])):
            for p in range(0,num_classes):
                if(trainLabelVector[i]!=p):   
                    for j in range(0,num_classes):
                        if(j==trainLabelVector[i]):
                            fullXmatrix[count5,(j*(X_data.shape[1]+1)):((j+1)*(X_data.shape[1]+1))-1] = X[i,:]
                            fullXmatrix[count5,((j+1)*(X_data.shape[1]+1))-1] = 1.0
                        elif(j==p):
                            fullXmatrix[count5,(j*(X_data.shape[1]+1)):((j+1)*(X_data.shape[1]+1))-1] = -1*X[i,:]
                            fullXmatrix[count5,((j+1)*(X_data.shape[1]+1))-1] = -1.0
                    count5 += 1
                    #print(count5)
        #print(fullXmatrix.shape)
        m = fullXmatrix.shape[0]
        M1 = sparse.csr_matrix(fullXmatrix) 
        IntermXmatrix = np.zeros(((X_data.shape[0]*(num_classes-1)),num_classes))
        #if(kernel=='polynomial'):   
        #    H = sparse.csr_matrix.dot(M1,M1.T) 
        #    H = (1+H.todense())**degree   
            #H,IntermXmatrix = self.getKernelizedProduct(X_data,Y_label,kernel,sigma)
        if(kernel=='linear'):
            H = sparse.csr_matrix.dot(M1,M1.T)
        else:
            H,IntermXmatrix = self.getKernelizedProduct(X_data,Y_label,kernel,sigma,degree)        

        from cvxopt import matrix as cvxopt_matrix
        from cvxopt import solvers as cvxopt_solvers
        if(kernel=='linear'):
            P = cvxopt_matrix(H.todense())
        else:
            P = cvxopt_matrix(H)
        q = cvxopt_matrix(-1*np.ones((m, 1)))
        G = cvxopt_matrix(np.vstack((np.eye(m)*-1,np.eye(m))))
        h = cvxopt_matrix(np.hstack((np.zeros(m), np.ones(m) * C)))
        opts = {'maxiters' : 30,'solver':'mosek'}
        sol = cvxopt_solvers.qp(P, q, G, h ,options = opts)
        alphas2 = np.array(sol['x']) 
        #w = fullXmatrix.T@alphas2
        #wForEachLabel = {}
        #for i in range(0,num_classes):
            #wForEachLabel[i] = np.array_split(w,num_classes)[i]
        self.alphas2=alphas2
        self.fullXmatrix=fullXmatrix
        self.IntermXmatrix=IntermXmatrix
        
        #return alphas2,fullXmatrix,IntermXmatrix
    def predict(self,val_X_data):
    #def predict(train_X_data,val_X_data,IntermXmatrix,num_classes,alphas2,fullXmatrix,kernel,degree,sigma):
        num_classes = self.num_classes
        train_X_data = self.train_X_data
        kernel = self.kernel
        degree = self.degree
        sigma = self.sigma
        alphas2=self.alphas2
        fullXmatrix=self.fullXmatrix
        IntermXmatrix=self.IntermXmatrix

        predictedLabelVector = []
        if(kernel == 'linear'):
            w = fullXmatrix.T@alphas2
            wForEachLabel = {}
            for i in range(0,num_classes):
                wForEachLabel[i] = np.array_split(w,num_classes)[i]
            print('\n Calculating prediction values')
            for i in tqdm(range(0,val_X_data.shape[0])):
                scoresForEachLabel = []
                for j in range(0,len(wForEachLabel)):
                    score = (wForEachLabel[j].T@np.append(val_X_data[i,:],1.0).reshape(-1,1))
                    scoresForEachLabel.append(score)
                maxScoreLabel = np.argmax(scoresForEachLabel)
                predictedLabelVector.append(maxScoreLabel)
        elif(kernel == 'polynomial'):
            print('\n Calculating prediction values')
            for i in tqdm(range(0,val_X_data.shape[0])):
                #tempMatrix1 = np.zeros((num_classes*(val_X_data.shape[1]+1),num_classes))
                #for j in range(0,num_classes):
                #    tempMatrix1[j*(val_X_data.shape[1]+1):(j+1)*(val_X_data.shape[1]+1),j] = np.append(val_X_data[i,:],1.0)
                #scoresForEachLabel = (alphas2.T)@((1+(fullXmatrix@tempMatrix1))**degree)
                scoresForEachLabel = (alphas2.T)@(self.getKernelizedProductforPrediction(train_X_data,val_X_data,IntermXmatrix,i,num_classes,kernel,sigma,degree))
                maxScoreLabel = np.argmax(scoresForEachLabel)
                predictedLabelVector.append(maxScoreLabel)
        elif(kernel == 'rbf'):
            print('\n Calculating prediction values')
            for i in tqdm(range(0,val_X_data.shape[0])):
                #tempMatrix1 = np.zeros((num_classes*(val_X_data.shape[1]+1),num_classes))
                #for j in range(0,num_classes):
                    #tempMatrix1[j*(val_X_data.shape[1]+1):(j+1)*(val_X_data.shape[1]+1),j] = np.append(val_X_data[i,:],1.0)
                scoresForEachLabel = (alphas2.T)@(self.getKernelizedProductforPrediction(train_X_data,val_X_data,IntermXmatrix,i,num_classes,kernel,sigma,degree))
                maxScoreLabel = np.argmax(scoresForEachLabel)
                predictedLabelVector.append(maxScoreLabel)
        return predictedLabelVector
    def getKernelizedProductforPrediction(self,X_data,val_X_data,IntermXmatrix,currentValIndex,num_classes,kernel,sigma,degree):
        X = X_data
        m = (X_data.shape[0]*(num_classes-1))
        H = np.zeros((m,num_classes))    
        matrix2 = (currentValIndex+1)*np.eye(num_classes)
        tempCoeff = (1/(X.shape[1]+1))
        
        if(kernel == 'polynomial'): 
            for i in (range(0,m)):
                for j in range(0,num_classes):
                    count1 = 0
                    for k in range(0,num_classes):
                        if(count1==2):
                            break
                        else:
                            temp3 = int(IntermXmatrix[i,k])
                            temp4 = int(matrix2[k,j])
                            if(temp3>0):
                                count1+=1
                                if(temp4>0):                             
                                    H[i,j] += (1+((1/tempCoeff)*np.dot(np.append(X[temp3-1,:],1.0),np.append(val_X_data[temp4-1,:],1.0))))**degree
                            elif(temp3<0):
                                count1+=1
                                if(temp4>0):
                                    H[i,j] += (1+((1/tempCoeff)*np.dot(np.append(val_X_data[temp4-1,:],1.0),np.append(-1*X[-temp3-1,:],-1.0))))**degree
        

        elif(kernel == 'rbf'): 
            for i in (range(0,m)):
                for j in range(0,num_classes):
                    count1 = 0
                    for k in range(0,num_classes):
                        if(count1==2):
                            break
                        else:
                            temp3 = int(IntermXmatrix[i,k])
                            temp4 = int(matrix2[k,j])
                            if(temp3>0):
                                count1+=1
                                if(temp4>0):                             
                                    H[i,j] += np.exp(-1*(np.linalg.norm(X[temp3-1,:]-val_X_data[temp4-1,:])**2)/(2*sigma**2))                        
                            elif(temp3<0):
                                count1+=1
                                if(temp4>0):
                                    H[i,j] += np.exp(-1*(np.linalg.norm(np.append(val_X_data[temp4-1,:],1.0)-np.append(-1*X[-temp3-1,:],-1.0))**2)/(2*sigma**2))                    

        return H      
    def getKernelizedProduct(self,X_data,Y_label,kernel,sigma,degree):
        X = X_data
        trainLabelVector = Y_label
        num_classes = len(np.unique(Y_label))
        IntermXmatrix = np.zeros(((X_data.shape[0]*(num_classes-1)),num_classes))
        count6 = 0
        m = (X_data.shape[0]*(num_classes-1))
        nonZeroColumnIndicesAndValuesInIntermXmatrix = []# np.zeros((m,4)) 
        print('\n Calculating Kernalized Product')
        
        for i in tqdm(range(0,X.shape[0])):
            for p in range(0,num_classes):
                if(trainLabelVector[i]!=p):   
                    tempArray = []
                    for j in range(0,num_classes):
                        if(j==trainLabelVector[i]):
                            tempArray.append(j)
                            tempArray.append(i+1)
                            IntermXmatrix[count6,j] = i+1
                        elif(j==p):
                            tempArray.append(j)
                            tempArray.append(-(i+1))
                            IntermXmatrix[count6,j] = -(i+1)
                    count6 += 1
                    nonZeroColumnIndicesAndValuesInIntermXmatrix.append(tempArray)
                    
        #nonZeroColumnIndicesAndValuesInIntermXmatrix = np.array(nonZeroColumnIndicesAndValuesInIntermXmatrix)
        matrix2 = IntermXmatrix.T
        M2 = scipy.sparse.coo_matrix(IntermXmatrix)
        M3 = scipy.sparse.coo_matrix(IntermXmatrix.T)
        m = (X_data.shape[0]*(num_classes-1))
        H = np.zeros((m,m))
        tempCoeff = (1/(X.shape[1]+1))
        #tempCoeff = 2
        
        
        if(kernel == 'rbf'):
            matrix2 = matrix2.tolist()
            #X = X.tolist()
            print('\n Calculating Kernalized Product')
            for i in tqdm(range(0,m)):
                for j in range(0,i+1):
                    #nonZeroColumnIndicesAndValuesInIntermXmatrix[i][0]
                    #nonZeroIndicesLocation = np.nonzero(IntermXmatrix[i,:])
                    #temp8 = IntermXmatrix[i,:][nonZeroIndicesLocation][0]
                    temp8 = nonZeroColumnIndicesAndValuesInIntermXmatrix[i][1]
                    #temp9 = IntermXmatrix[i,:][nonZeroIndicesLocation][1]
                    temp9 =nonZeroColumnIndicesAndValuesInIntermXmatrix[i][3]
                    #locOftemp8 = nonZeroIndicesLocation[0]
                    locOftemp8 = nonZeroColumnIndicesAndValuesInIntermXmatrix[i][0]
                    #locOftemp9 = nonZeroIndicesLocation[1]
                    locOftemp9 = nonZeroColumnIndicesAndValuesInIntermXmatrix[i][2]
                    temp10 = matrix2[locOftemp8][j]
                    temp11 = matrix2[locOftemp9][j]
                    #temp10zero = 1
                    #temp11zero = 1
                    #if(temp10 == 0):
                        #temp10zero = 0
                    #if(temp11 == 0):
                        #temp11zero = 0
                    temp8Pos = 1
                    temp9Pos = 1
                    temp10Pos = 1
                    temp11Pos = 1
                    
                    if(temp8 < 0):
                        temp8Pos = -1
                    if(temp9 < 0):
                        temp9Pos = -1
                    if(temp10 < 0):
                        temp10Pos = -1
                    if(temp11 < 0):
                        temp11Pos = -1
                        
                    if(temp10 != 0):
                        #H[i,j] += (2.718281828459045)**(-1*normsquare(np.append(temp8Pos*X[int(temp8Pos*temp8)-1,:],temp8Pos),np.append(temp10Pos*X[int(temp10Pos*temp10)-1,:],temp10Pos))/(2*sigma**2))                       
                        H[i,j] += (2.718281828459045)**(-1*(np.linalg.norm(np.append(temp8Pos*X[int(temp8Pos*temp8)-1,:],temp8Pos)-np.append(temp10Pos*X[int(temp10Pos*temp10)-1,:],temp10Pos))**2)/(2*sigma**2))
                    if(temp11 != 0):
                        #H[i,j] += (2.718281828459045)**(-1*normsquare(np.append(temp9Pos*X[int(temp9Pos*temp9)-1,:],temp9Pos),np.append(temp11Pos*X[int(temp11Pos*temp11)-1,:],temp11Pos))/(2*sigma**2))
                        H[i,j] += (2.718281828459045)** (-1*(np.linalg.norm(np.append(temp9Pos*X[int(temp9Pos*temp9)-1,:],temp9Pos)-np.append(temp11Pos*X[int(temp11Pos*temp11)-1,:],temp11Pos))**2)/(2*sigma**2))    

        
        elif(kernel == 'polynomial'):
            print('\n Calculating Kernalized Product')
            for i in tqdm(range(0,m)):
                for j in range(0,i+1):
                    count1 = 0
                    #count2 = 0
                    for k in range(0,IntermXmatrix.shape[1]):
                        if(count1==2):
                            break
                        else:
                            temp3 = int(IntermXmatrix[i,k])
                            temp4 = int(matrix2[k,j])
                            #print(temp3,temp4)
                            if(temp3>0):
                                count1+=1
                                if(temp4>0): 
                                    H[i,j] += ((1+((1/tempCoeff)*np.dot(np.append(X[temp3-1,:],1.0),np.append(X[temp4-1,:],1.0))))**degree)
                                elif(temp4<0):                                    
                                    H[i,j] += ((1+((1/tempCoeff)*np.dot(np.append(X[temp3-1,:],1.0),np.append(-1*X[-temp4-1,:],-1.0))))**degree)
                            elif(temp3<0):
                                count1+=1
                                if(temp4>0):
                                    H[i,j] += ((1+((1/tempCoeff)*np.dot(np.append(X[temp4-1,:],1.0),np.append(-1*X[-temp3-1,:],-1.0))))**degree)
                                elif(temp4<0):
                                    H[i,j] += ((1+((1/tempCoeff)*np.dot(np.append(-1*X[-temp4-1,:],-1.0),np.append(-1*X[-temp3-1,:],-1.0))))**degree)
              

        elif(kernel == 'kartheek'):
            print('\n Calculating Kernalized Product')
            for i in tqdm(range(0,m)):
                for j in range(0,i+1):
                    count1 = 0
                    #count2 = 0
                    for k in range(0,IntermXmatrix.shape[1]):
                        if(count1==2):
                            break
                        else:
                            temp3 = int(IntermXmatrix[i,k])
                            temp4 = int(matrix2[k,j])
                            #print(temp3,temp4)
                            if(temp3>0):
                                count1+=1
                                if(temp4>0):                             
                                    H[i,j] += np.exp(-1*(np.linalg.norm(X[temp3-1,:]-X[temp4-1,:])**2)/(2*sigma**2))
                                elif(temp4<0):
                                    H[i,j] += np.exp(-1*(np.linalg.norm(np.append(X[temp3-1,:],1.0)-np.append(-1*X[-temp4-1,:],-1.0))**2)/(2*sigma**2))
                            elif(temp3<0):
                                count1+=1
                                if(temp4>0):
                                    H[i,j] += np.exp(-1*(np.linalg.norm(np.append(X[temp4-1,:],1.0)-np.append(-1*X[-temp3-1,:],-1.0))**2)/(2*sigma**2))                    
                                elif(temp4<0):
                                    H[i,j] += np.exp(-1*(np.linalg.norm(-1*X[-temp3-1,:]+X[-temp4-1,:])**2)/(2*sigma**2))
        print('\n Calculating Kernalized Product')
        for i in tqdm(range(0,m)):
            for j in range(0,i+1):
                if(j>i):
                    H[i,j]=H[j,i]                 
        #print(H.shape)        

        return H,IntermXmatrix

WINE DATASET

In [101]:
from sklearn.datasets import load_wine
Givendata = load_wine()
ran = np.random.rand(len(Givendata['data'])) < 0.8
train_X_data = Givendata['data'][ran]
train_Y_data = Givendata['target'][ran]
val_X_data = Givendata['data'][~ran]
val_Y_data = Givendata['target'][~ran]
kernel = 'rbf'
degree = 4
sigma = 0.4
C = 10
a1 = SMCSVM()
#a1.fit()
start_time = time.time()
a1.fit(train_X_data,train_Y_data,C,kernel,degree,sigma)
print('\n Time taken for training in seconds: ', time.time()-start_time )
predictedLabels = a1.predict(train_X_data)
conMatrix = pd.crosstab(np.array(predictedLabels).ravel(),train_Y_data.reshape(-1,1).ravel())
conMatrix


 Calculating fullXmatrix



 Calculating Kernalized Product



 Calculating Kernalized Product



 Calculating Kernalized Product


     pcost       dcost       gap    pres   dres
 0:  1.4472e+03 -4.9364e+03  6e+03  3e-17  1e-15
 1:  1.6230e+02 -4.8632e+02  6e+02  2e-16  1e-15
 2: -3.2852e+01 -1.2128e+02  9e+01  2e-16  8e-16
 3: -4.7440e+01 -5.4771e+01  7e+00  2e-16  2e-16
 4: -4.7667e+01 -4.7769e+01  1e-01  1e-16  9e-17
 5: -4.7667e+01 -4.7668e+01  1e-03  3e-16  1e-16
 6: -4.7667e+01 -4.7667e+01  1e-05  2e-16  1e-16
Optimal solution found.

 Time taken for training in seconds:  2.875321388244629

 Calculating prediction values


col_0,0,1,2
row_0,,,
0,50,0,0
1,0,52,0
2,0,0,41


IRIS DATASET

In [96]:
from sklearn.datasets import load_iris
Givendata =load_iris()
ran = np.random.rand(len(Givendata['data'])) < 0.8
train_X_data = Givendata['data'][ran]
train_Y_data = Givendata['target'][ran]
val_X_data = Givendata['data'][~ran]
val_Y_data = Givendata['target'][~ran]
kernel = 'rbf'
degree = 4
sigma = 0.4
C = 10
a1 = SMCSVM()
#a1.fit()
start_time = time.time()
a1.fit(train_X_data,train_Y_data,C,kernel,degree,sigma)
print('\n Time taken for training in seconds: ', time.time()-start_time )
predictedLabels = a1.predict(train_X_data)
conMatrix = pd.crosstab(np.array(predictedLabels).ravel(),train_Y_data.reshape(-1,1).ravel())
conMatrix


 Calculating fullXmatrix



 Calculating Kernalized Product



 Calculating Kernalized Product



 Calculating Kernalized Product


     pcost       dcost       gap    pres   dres
 0:  4.5452e+02 -5.1798e+03  1e+04  2e-01  4e-15
 1:  2.4747e+02 -5.1588e+02  8e+02  6e-03  2e-15
 2:  2.6061e+01 -7.5205e+01  1e+02  2e-16  3e-15
 3: -4.9037e+00 -1.8381e+01  1e+01  2e-16  1e-15
 4: -7.5505e+00 -9.0699e+00  2e+00  2e-16  4e-16
 5: -7.8953e+00 -8.2773e+00  4e-01  2e-16  2e-16
 6: -8.0225e+00 -8.0994e+00  8e-02  1e-16  2e-16
 7: -8.0435e+00 -8.0575e+00  1e-02  2e-16  2e-16
 8: -8.0480e+00 -8.0486e+00  6e-04  2e-16  2e-16
 9: -8.0482e+00 -8.0482e+00  3e-05  2e-16  2e-16
10: -8.0482e+00 -8.0482e+00  5e-07  2e-16  2e-16
Optimal solution found.

 Time taken for training in seconds:  1.9268863201141357

 Calculating prediction values


col_0,0,1,2
row_0,,,
0,37,0,0
1,0,42,1
2,0,1,34


CANCER DATASET

In [106]:
from sklearn.datasets import load_breast_cancer
Givendata = load_breast_cancer()
ran = np.random.rand(len(Givendata['data'])) < 0.8
train_X_data = Givendata['data'][ran]
train_Y_data = Givendata['target'][ran]
val_X_data = Givendata['data'][~ran]
val_Y_data = Givendata['target'][~ran]
kernel = 'rbf'
degree = 4
sigma = 0.4
C = 10
a1 = SMCSVM()
#a1.fit()
start_time = time.time()
a1.fit(train_X_data,train_Y_data,C,kernel,degree,sigma)
print('\n Time taken for training in seconds: ', time.time()-start_time )
predictedLabels = a1.predict(train_X_data)
conMatrix = pd.crosstab(np.array(predictedLabels).ravel(),train_Y_data.reshape(-1,1).ravel())
conMatrix


 Calculating fullXmatrix



 Calculating Kernalized Product



 Calculating Kernalized Product



 Calculating Kernalized Product


     pcost       dcost       gap    pres   dres
 0:  2.1512e+03 -7.8504e+03  1e+04  0e+00  0e+00
 1:  2.0363e+02 -8.4729e+02  1e+03  9e-17  9e-16
 2: -9.2760e+01 -2.3273e+02  1e+02  4e-16  2e-16
 3: -1.1159e+02 -1.2086e+02  9e+00  2e-17  7e-18
 4: -1.1175e+02 -1.1186e+02  1e-01  3e-17  3e-17
 5: -1.1175e+02 -1.1175e+02  1e-03  3e-16  4e-19
 6: -1.1175e+02 -1.1175e+02  1e-05  5e-17  1e-18
Optimal solution found.

 Time taken for training in seconds:  6.27635931968689

 Calculating prediction values


col_0,0,1
row_0,,
0,165,0
1,0,282


DIGITS DATASET

In [107]:
from sklearn.datasets import load_digits
data1 = load_digits()
Givendata = dimensionality_reduction(data1['data'],data1['target'],save=False)
ran = np.random.rand(len(Givendata)) < 0.8
train_X_data = Givendata[ran]
train_Y_data = data1['target'][ran]
val_X_data = Givendata[~ran]
val_Y_data = data1['target'][~ran]
kernel = 'rbf'
degree = 4
sigma = 0.4
C = 10
a1 = SMCSVM()
#a1.fit()
start_time = time.time()
a1.fit(train_X_data,train_Y_data,C,kernel,degree,sigma)
print('\n Time taken for training in seconds: ', time.time()-start_time )
predictedLabels = a1.predict(train_X_data)
conMatrix = pd.crosstab(np.array(predictedLabels).ravel(),train_Y_data.reshape(-1,1).ravel())
conMatrix

reduced dimensions are 
(1797, 15)

 Calculating fullXmatrix



 Calculating Kernalized Product



 Calculating Kernalized Product


KeyboardInterrupt: 

In [102]:
def plot_decision_regions(classifier,train_X_data,train_Y_data,resolution=1):
   # setup marker generator and color map
   markers = ('s', 'x', 'o', '^', 'v')
   colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
   X = train_X_data
   y = train_Y_data
   cmap = ListedColormap(colors[:len(np.unique(train_Y_data))])
   # plot the decision surface
   MinFirstAttr, MaxFirstAttr = train_X_data[:, 0].min() - 1 , train_X_data[:, 0].max() + 1
   MinSecAttr, MaxSecAttr = train_X_data[:, 1].min() - 1, train_X_data[:, 1].max() + 1
   MeshGridPtsFirstAttr, MeshGridPtsSecAttr = np.meshgrid(np.arange(x1_min, x1_max, resolution),np.arange(x2_min, x2_max, resolution))
   Z = classifier.predict(np.array([MeshGridPtsFirstAttr.ravel(), MeshGridPtsSecAttr.ravel()]).T)
   Z = np.array(Z) 
   Z = Z.reshape(MeshGridPtsFirstAttr.shape)
   plt.contourf(MeshGridPtsFirstAttr, MeshGridPtsSecAttr, Z, alpha=0.1, cmap=cmap)
   plt.xlim(MeshGridPtsFirstAttr.min(), MeshGridPtsFirstAttr.max())
   plt.ylim(MeshGridPtsSecAttr.min(), MeshGridPtsSecAttr.max())
   for index, cl in enumerate(np.unique(train_Y_data)):
      plt.scatter(x=train_X_data[train_Y_data == cl, 0], y=train_X_data[train_Y_data == cl, 1],alpha=0.8, c=cmap(idx),marker=markers[index], label=cl)
   plt.legend(loc='upper left')
   plt.show()

In [13]:
classifier = a1
plot_decision_regions(classifier,train_X_data,train_Y_data,resolution=0.1)


 Calculating prediction values


ValueError: operands could not be broadcast together with shapes (30,) (2,) 

In [ ]:
def performCrossValidation(X1,Y1,num_folds,classifier,kernel='rbf',degree=1,sigma=1.0):
    C = 10
    fullXmatrix1 = X1
    fullYvector1 = Y1
    from sklearn.model_selection import KFold
    kf = KFold(n_splits=num_folds)
    kf.get_n_splits(fullXmatrix1)
    consideredSigmaValues = [0.1,0.3,0.4,0.5,0.6,0.7,1.0]
    consideredCValues = [0,50,100]
    consideredKernelValues = ['linear','polynomial','rbf']
    fig, ax = plt.subplots(figsize=(15,10))
    for kernel in consideredKernelValues:
        for C in consideredCValues:
            crossValAvgErrorForEachSigma = []
            crossValVarOfErrorForEachSigma = []
            
            for sigma in consideredSigmaValues:#, 1e-4 ,1e-3, 1e-2, 0.1, 1, 10, 100 ,1000]:
                ValErrorForEachFold = []
                fold = 0
                for train_index, test_index in kf.split(fullXmatrix1):
                    print('fold is '+str(fold))
                    fold += 1
                    #print("TRAIN:", train_index, "TEST:", test_index)
                    X_train, X_test = fullXmatrix1[train_index], fullXmatrix1[test_index]
                    y_train, y_test = fullYvector1[train_index], fullYvector1[test_index]
                    trainXmatrix = X_train
                    ValXmatrix = X_test
                    trainYvector = y_train
                    valYvector = y_test
                    numOfSamples,numOfdimPlusOne = trainXmatrix.shape
                    classifier.fit(X_train, y_train,C,kernel,degree,sigma)
                    #classifier.fit(X_train, y_train,lamda,kernel,degree,sigma)
                    #classifier.fit(X_train, y_train,C,lamda,degree,sigma)
                    predictedLabels = classifier.predict(X_test)
                    numOfCorrectlyPredictedLabels = 0
                    for index in range(0,len(valYvector)):
                        if(valYvector[index] == predictedLabels[index]):
                            numOfCorrectlyPredictedLabels += 1
                    accuracy = numOfCorrectlyPredictedLabels/len(valYvector)
                    error = 1- accuracy
                    ValErrorForEachFold.append(error)
                crossValAvgErrorForEachSigma.append(np.mean(ValErrorForEachFold))
                crossValVarOfErrorForEachSigma.append(np.std(ValErrorForEachFold))
            
            #for i in numOfdimPlusOne:#[:-1]
            #for i in lamdavalues:
            #ax.plot(np.arange(0.8,0.91,0.01),crossValAvgErrorForEachLamda, label= 'Bias Estimate')
            ax.plot(consideredSigmaValues,crossValAvgErrorForEachSigma, label= 'Bias Estimate for Kernel= '+str(kernel)+'C='+str(C))
            ax.plot(consideredSigmaValues,crossValVarOfErrorForEachSigma, label= 'Variance Estimate for Kernel= '+str(kernel)+'C='+str(C))
            if((len(consideredKernelValues)==1) and (len(consideredCValues)==1) and (len(consideredSigmaValues)==1)):
                print('cross validation error estimate is '+str(crossValAvgErrorForEachSigma))
            ax.legend(loc=0) # upper left corner
            ax.set_xlabel('Sigma')
            ax.set_ylabel('values')
            ax.set_title('(using cross validation) Bias & Variance Estimate for different hyperparameter values');
    plt.show()

In [30]:
x = np.array([3, 0, 0, 5, 8])
x
t = np.nonzero(x)
x[t][2]

8

In [32]:
x = np.array([[3, 0, 0], [0, 4, 0], [5, 6, 0]])
x[2,:]

array([5, 6, 0])